In [15]:
!nvidia-smi

Fri Dec  5 15:56:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             35W /   70W |    6216MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
%%capture
!pip install unsloth

In [8]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
messages = [
    {
        "role": "system",
        "content": "Você é um assistente que gera apenas código SQL. Responda apenas com o código, sem explicações."
    },
    {
        "role": "user",
        "content": "Gere uma query SQL para contar quantas pessoas tem mais de 56 anos na tabela users (campo age)."
    }
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Você é um assistente que gera apenas código SQL. Responda apenas com o código, sem explicações.<|eot_id|><|start_header_id|>user<|end_header_id|>

Gere uma query SQL para contar quantas pessoas tem mais de 56 anos na tabela users (campo age).<|eot_id|><|start_header_id|>assistant<|end_header_id|>


CPU times: user 1.08 ms, sys: 0 ns, total: 1.08 ms
Wall time: 1.03 ms


In [12]:
tokenized_prompt = tokenizer(prompt, return_tensors='pt').to('cuda')

FastLanguageModel.for_inference(model)
text_streamer = TextStreamer(tokenizer)

In [14]:
%%time
response = model.generate(
    **tokenized_prompt,
    streamer=text_streamer,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.15,
    no_repeat_ngram_size=3,
    do_sample=False,
    temperature=0.0
)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Você é um assistente que gera apenas código SQL. Responda apenas com o código, sem explicações.<|eot_id|><|start_header_id|>user<|end_header_id|>

Gere uma query SQL para contar quantas pessoas tem mais de 56 anos na tabela users (campo age).<|eot_id|><|start_header_id|>assistant<|end_header_id|>

SELECT COUNT(*) FROM users WHERE age > 56;<|eot_id|>
CPU times: user 882 ms, sys: 7.05 ms, total: 890 ms
Wall time: 892 ms
